# PRI Project

## Data Processing

Write something about the project here.

In [ ]:
import pandas as pd
import re

Write something about data 1 here

In [ ]:
# Read data 1 csv and drop the unnecessary data
jobs_data_1 = pd.read_csv('data/allJobs.csv')
jobs_data_1.drop(columns=['Date-Posted'],inplace=True)
jobs_data_1.dropna()
jobs_data_1.rename(columns = {'Company':'company','Job-Title':'title','Job-Type':'work-type','Salary':'salary','Location':'location','Description':'description'}, inplace = True)
jobs_data_1.head()

Write something about data 2 hera

In [ ]:
# Read data 2 csv and drop the unnecessary data
jobs_data_2 = pd.read_csv('data/job_postings.csv')
jobs_data_2.drop(columns=['max_salary','min_salary','job_id','expiry','closed_time','applies','currency','compensation_type','original_listed_time','remote_allowed','views','job_posting_url','application_url','application_type','formatted_experience_level','skills_desc','listed_time','posting_domain','sponsored','work_type'],inplace=True)
jobs_data_2.rename(columns = {'med_salary':'salary','formatted_work_type':'work-type'}, inplace = True)
jobs_data_2.head()

Write something about the salary in jobs_data_1 here

In [ ]:
def extract_salary_and_pay_period(salary_string):
    salary_string = str(salary_string)
    if 'a year' in salary_string:
        pay_period = 'YEARLY'
        match = re.search(r'\$([\d,.]+[kK]?)-?([\d,.]+[kK]?)?', salary_string)
        if match:
            low_value = match.group(1).replace('.', '').replace(',', '').replace('k', '000').replace('K', '000')
            high_value = match.group(2).replace('.', '').replace(',', '').replace('k', '000').replace('K', '000') if match.group(2) else low_value
            salary = (float(low_value) + float(high_value)) / 2
        else:
            match = re.search(r'\$([\d,.]+[kK]?)', salary_string)
            if match:
                salary = float(match.group(1).replace('.', '').replace(',', '').replace('k', '000').replace('K', '000'))
            else:
                salary = None
    elif 'a month' in salary_string:
        pay_period = 'MONTHLY'
        match = re.findall(r'\$([\d,.]+[kK]?)', salary_string)
        if match:
            salary_range = [(float(m.replace('.', '').replace(',', '').replace('k', '000').replace('K', '000'))) for m in match]
            salary = sum(salary_range) / len(salary_range)
        else:
            salary = None
    elif 'an hour' in salary_string:
        pay_period = 'HOURLY'
        match = re.search(r'\$([\d,.]+[kK]?)', salary_string)
        if match:
            salary = float(match.group(1).replace('.', '').replace(',', '').replace('k', '000').replace('K', '000'))
        else:
            salary = None
    else:
        pay_period = None
        salary = None
    return salary, pay_period

... apply the function

In [ ]:
# Split the salary collumn into salary and pay period
jobs_data_1['salary'], jobs_data_1['pay_period'] = zip(*jobs_data_1['salary'].apply(extract_salary_and_pay_period))
# Remove outliers 
jobs_data_1 = pd.concat([jobs_data_1[jobs_data_1['salary'] < 1000000], jobs_data_1[jobs_data_1['salary'].isnull()]])
jobs_data_1.head()

.. add company data

In [ ]:
# Create a new dataframe for company data
company_data_1 = jobs_data_1["company"].drop_duplicates().reset_index(drop=True).reset_index()
company_data_1.columns = ['company_id', 'company']
company_data_1['company_id'] = company_data_1['company_id'] + 3700152515

jobs_data_1 = pd.merge(jobs_data_1, company_data_1, on='company', how='left')
jobs_data_1.drop(columns=['company'],inplace=True)
jobs_data_1['work-type'].replace({'Contractor': 'Contract'}, inplace=True)

company_data_2 = pd.read_csv('data/companies.csv')
company_data_2.drop(columns=['description',	'company_size',	'state'	,'country'	,'city',	'zip_code',	'address',	'url'],inplace=True)
company_data_2.rename(columns = {'name':'company'}, inplace = True)

company_data = pd.concat([company_data_1,company_data_2])
company_data

...merge dataframes

In [ ]:
# Merge dataframes
jobs_data = pd.concat([jobs_data_1, jobs_data_2], ignore_index=True)
jobs_data.head()

... write something about country data

In [ ]:
# Read countries csv and drop the unnecessary data
countries_data = pd.read_csv('data/countries.csv')
countries_data = countries_data[['name','alpha-2','region','sub-region']]
countries_data.head()

... extract city and country data

In [ ]:
def extract_city_and_country(location_string, country_list):
    splited_location = location_string.split(',')
    first_word = splited_location[0]
    last_word = splited_location[-1][1:]
    country_names = country_list['name'].values
    country_alpha = country_list['alpha-2'].values

    if first_word in country_names:
        region = country_list[country_list['name'] == first_word]['region'].iloc[0]
        return None, first_word, region
    elif first_word in country_alpha:
        country = country_list[country_list['alpha-2'] == first_word]['name'].iloc[0]
        region = country_list[country_list['alpha-2'] == first_word]['region'].iloc[0]
        return None, country, region
    elif last_word in country_names:
        region = country_list[country_list['name'] == last_word]['region'].iloc[0]
        return first_word, last_word, region
    elif last_word in country_alpha:
        country = country_list[country_list['alpha-2'] == last_word]['name'].iloc[0]
        region = country_list[country_list['alpha-2'] == last_word]['region'].iloc[0]
        return first_word, country, region
    elif len(splited_location) > 0:
        return first_word, 'United States of America', 'Americas'
    return None, None, None

... aply function

In [ ]:
# Create location table
jobs_location_data = pd.DataFrame(jobs_data['location'], columns=['location'])
jobs_location_data['city'], jobs_location_data['country'], jobs_location_data['region'] = zip(*jobs_location_data['location'].apply(lambda x: extract_city_and_country(x, countries_data)))
jobs_location_data.drop_duplicates(inplace=True)
jobs_location_data.reset_index(drop=True, inplace=True)
jobs_location_data.insert(0, 'location_id', range(1, 1 + len(jobs_location_data)))

jobs_data = pd.merge(jobs_data, jobs_location_data[['location', 'location_id']], on='location', how='left')
jobs_data.drop(columns=['location'], inplace=True)

jobs_location_data.drop(columns=['location'], inplace=True)
jobs_data.head()

... graphic 8K HDR 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

jobs_data['pay_period'].value_counts().plot(kind='bar')
plt.xlabel('Pay-Period')
plt.ylabel('Job postings')
plt.title('Job postings by Pay-Period')

In [ ]:
jobs_data_final_copy = jobs_data.copy().reset_index()

for idx ,job in jobs_data_final_copy.iterrows():
    if job["pay_period"] == "MONTHLY":
        jobs_data_final_copy.at[idx ,"salary"] *= 12
    elif job["pay_period"] == "HOURLY":
        jobs_data_final_copy.at[idx ,"salary"] *= 1810

average_salary_by_work_type = jobs_data_final_copy.groupby('work-type')['salary'].mean().reset_index()
plt.figure(figsize=(10 ,6))  
plt.bar(average_salary_by_work_type["work-type"] ,average_salary_by_work_type["salary"] ,color='skyblue')
plt.xlabel('Work Type')
plt.ylabel('Average Salary')
plt.title('Average Salary by Work Type')
plt.show()

In [ ]:
jobs_data_final_copy = pd.merge(jobs_data_final_copy, jobs_location_data, on='location_id', how='left')
average_salary_by_region = jobs_data_final_copy.groupby('region')['salary'].mean().reset_index()
plt.figure(figsize=(10 ,6))
plt.bar(average_salary_by_region["region"] ,average_salary_by_region["salary"] ,color='skyblue')
plt.xlabel('Region')
plt.ylabel('Average Salary')
plt.title('Average Salary by Region')
plt.show()

In [ ]:
jobs_data_final_copy['region'].value_counts().plot(kind='pie',explode=(.1, 0, 0, 0,0),shadow=True,autopct='%1.1f%%',ylabel="",title="Job postings by World Region")
jobs_data_final_copy.head()

## Information Retrieval

In [ ]:
# Initialize docker container
!powershell -command "docker compose down"
!powershell -command "docker compose up -d"

In [ ]:
# Wait until solr is ready
import requests
import time

url = 'http://localhost:8983/solr'

while True:
    try:
        response = requests.get(url)
        if response.status_code == 200:
            print('Solr is ready.')
            break
    except requests.exceptions.RequestException as e:
        print('Solr is not ready yet, waiting...')
        time.sleep(2)  # wait for 10 seconds before trying again

### Schema

In [ ]:
schema_path = 'solr/jobs_schema.json'

with open(schema_path, 'rb') as f:
    schema_data = f.read()

headers = {'Content-type': 'application/json'}

r = requests.post(url, data=schema_data, headers=headers)

if r.status_code == 200:
    print('Schema updated successfully')
else:
    print('Error updating schema')
    print(r.text)

### Insert data

In [54]:
import pysolr

solr = pysolr.Solr('http://localhost:8983/solr/mycore', timeout=10)

all_data = pd.merge(jobs_data, jobs_location_data, on='location_id', how='left')
all_data = pd.merge(all_data, company_data, on='company_id', how='left')
all_data.drop(columns=['location_id','company_id'],inplace=True)
all_data_doc = all_data.to_dict(orient='records')

chunk_size = 2000  # adjust this value based on your needs

# Split data into chunks and add them to Solr
for i in range(0, len(all_data_doc), chunk_size):
    chunk = all_data_doc[i:i+chunk_size]
    solr.add(chunk)
    solr.commit()

### Query 1 - Administration jobs in education establishments

In [ ]:
# Simple query
sq1 = solr.search(q='(company:university OR company:school OR company:college) AND (title:administration OR description:administration)', 
                sort='score desc', rows='10')

for hit in sq1:
    print(hit)

In [ ]:
# Boosting query
bq1 = solr.search(q='(company:university OR company:school OR company:college) AND (title:administration^4 OR description:administration)', 
                sort='score desc', rows='10')
for hit in bq1:
    print(hit)

### Query 2 - Jobs as part of a team in medical field

In [ ]:
# Simple query
sq2 = solr.search(q='(title:medical OR description:medical) AND (title:team OR description:team)', 
                sort='score desc', rows='10')

for hit in sq2:
    print(hit)

In [ ]:
# Boosting query
bq2 = solr.search(q='(title:medical^10 OR description:medical^5) AND (title:team OR description:team)', 
                sort='score desc', rows='10')

for hit in bq2:
    print(hit)

### Query 3 - Data engineering jobs with 3 year experience required

In [ ]:
# Simple query
sq3 = solr.search(q='(title:data description:data title:engineering description:engineering) AND description:"3 year experience"', 
                sort='score desc', rows='10')

for hit in sq3:
    print(hit)

In [ ]:
# Boosting query
bq3 = solr.search(q=' (title:data^5 description:data title:engineering^5 description:engineering) AND description:"3 year experience"~10', 
                sort='score desc', rows='10')

for hit in bq3:
    print(hit)

### Query 4 - Non-contract jobs, paid by hour, not computation related

In [ ]:
# Simple query
sq4 = solr.search(q='(-work-type:Contract AND pay_period:HOURLY AND -title:computation AND -description:computation)', 
                sort='score desc', rows='10')

for hit in sq4:
    print(hit)

In [ ]:
# Boosting query
sq4 = solr.search(q='(-work-type:Contract AND pay_period:HOURLY^5 AND -title:computation AND -description:computation)', 
                sort='score desc', rows='10')

for hit in sq4:
    print(hit)